In [1]:
import os
import numpy as np
import nibabel as nib
import ants

ModuleNotFoundError: No module named 'ants'

In [ ]:
dir = 'C:/Users/jcsimon/Documents/Lab/Stanford/Data/Bruker/20231117'
files = os.listdir(dir)
files = [f for f in files if f.endswith('.nii')]

image1 = ants.from_numpy(nib.load(dir + files[0]))
image2 = ants.from_numpy(nib.load(dir + files[1]))

grad_step = 0.2
flow_sigma = 3
total_sigma = 0
syn_sampling = 32
type_of_transform = 'Rigid'

moco = ants.registration(image1,image2,type_of_transform=type_of_transform,grad_step=grad_step, flow_sigma=flow_sigma,total_sigma=total_sigma)
    #syn_sampling=syn_sampling)

fwdtransforms = moco['fwdtransforms']

